<a href="https://colab.research.google.com/github/neel13062003/Egniol_Task/blob/master/FacebookClean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
data1 = pd.read_csv('1.csv')

In [5]:
# Select only the columns you want
filtered_data1 = data1[['created_time', 'ad_name', 'phone_number', 'company_name', 'email']]
print(filtered_data1)

    created_time                                  ad_name   phone_number  \
0        4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad  +919774939833   
1        4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad  +919689856127   
2        4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad  +919327631722   
3        4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad  +918485801000   
4        4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad  +919424130800   
..           ...                                      ...            ...   
220      4/18/23  Agriculture NAIF Funding 9 Lakh Data Ad  +919904230833   
221      4/18/23  Agriculture NAIF Funding 9 Lakh Data Ad  +919419034115   
222      4/18/23  Agriculture NAIF Funding 9 Lakh Data Ad  +919003027304   
223      4/18/23  Agriculture NAIF Funding 9 Lakh Data Ad  +919822141010   
224      4/18/23  Agriculture NAIF Funding 9 Lakh Data Ad  +918458976593   

              company_name                     email  
0                   Imphal      

In [11]:
# Select the required columns and create a new DataFrame explicitly with .copy()
filtered_data = data1[['created_time', 'ad_name', 'phone_number', 'company_name', 'email']].copy()

# Ensure the 'phone_number' column is text and clean non-digit characters
filtered_data.loc[:, 'phone_number'] = filtered_data['phone_number'].astype(str)  # Convert to string
filtered_data.loc[:, 'phone_number'] = filtered_data['phone_number'].str.replace(r'\D', '', regex=True)  # Remove non-digits

# Extract the last 10 digits of the phone number
filtered_data.loc[:, 'phone_number'] = filtered_data['phone_number'].str[-10:]



In [12]:
# Display the first few rows of the DataFrame to check the results
print(filtered_data.head())

  created_time                                  ad_name phone_number  \
0      4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad   9774939833   
1      4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad   9689856127   
2      4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad   9327631722   
3      4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad   8485801000   
4      4/21/23  Agriculture NAIF Funding 9 Lakh Data Ad   9424130800   

            company_name                     email  
0                 Imphal      rejobi2022@gmail.com  
1    baliraja bachat gat       zinjadenr@gmail.com  
2             rajal agri    neerajkadi06@gmail.com  
3  Akbani foods products  irfan.akbani84@gmail.com  
4               Director      hemantsai2@gmail.com  


In [14]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.2 MB/s eta 0:00:00


In [17]:
import mysql.connector
from mysql.connector import Error

def connect_to_database():
    try:
        connection = mysql.connector.connect(
            host='ls-a8f19dc65c2dfdb43dbb12300a1816cbf6fb7605.c2g9uvquz5lp.ap-south-1.rds.amazonaws.com',
            user='dbmasteruser',
            password='p)of$]?7`uA4]x..&TaaDL+<`8f2]Xx4',
            database='UrbanCompany'
        )
        if connection.is_connected():
            print('Connected to MySQL database')
            return connection
    except Error as e:
        print(f'Error connecting to MySQL database: {e}')
        return None


In [ ]:
def main():
    # Connect to the database
    connection = connect_to_database()
    if not connection:
        return
    # Define your SQL query
    query = """
    SELECT id,`from` as ContactNumber  FROM PLATFORM.text_waba
    WHERE text IN (SELECT text FROM PLATFORM.BUTTON)
    AND DATE(created_at) = "2024-04-13"
    LIMIT 5;
    """

    # Execute the SQL query
    records = execute_query(connection, query)
    if not records:
        return

    # Display the results
    print("Results:")
    for record in records:
        print(record)

if __name__ == "__main__":
    main()

In [23]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import json  # Import JSON to handle JSON data

# Function to read CSV data and process phone numbers
def process_csv(file_path):
    data = pd.read_csv(file_path)
    filtered_data = data[['created_time', 'ad_name', 'phone_number', 'company_name', 'email']].copy()
    # Clean phone numbers and extract the last 10 digits
    filtered_data['phone_number'] = filtered_data['phone_number'].astype(str).str.replace(r'\D', '', regex=True)
    filtered_data['phone_number'] = filtered_data['phone_number'].str[-10:]
    return filtered_data

# Function to connect to the MySQL database
def connect_to_database():
    try:
        connection = mysql.connector.connect(
            host='ls-a8f19dc65c2dfdb43dbb12300a1816cbf6fb7605.c2g9uvquz5lp.ap-south-1.rds.amazonaws.com',
            user='dbmasteruser',
            password='p)of$]?7`uA4]x..&TaaDL+<`8f2]Xx4',
            database='UrbanCompany'
        )
        if connection.is_connected():
            print('Connected to MySQL database')
            return connection
    except Error as e:
        print(f'Error connecting to MySQL database: {e}')
        return None

# Function to insert data into MySQL
def insert_data_to_db(connection, data):
    cursor = connection.cursor()
    insert_query = """
    INSERT INTO egniol_facebook (Created_At, Contact_Number, Company_Name, EmailID, Other)
    VALUES (%s, %s, %s, %s, %s)
    """
    # Insert each row of the filtered data into the database
    for _, row in data.iterrows():
        # Construct a JSON object for the "Other" column
        other_data = {
            'ad_name': row['ad_name'],  # Example of data to be included in JSON
        }
        cursor.execute(insert_query, (row['created_time'], row['phone_number'], row['company_name'], row['email'], json.dumps(other_data)))

    connection.commit()  # Commit the transaction
    cursor.close()

# Main function
def main():
    # Process CSV and get the data
    filtered_data = process_csv('1.csv')

    # Connect to the MySQL database
    connection = connect_to_database()
    if not connection:
        return

    # Insert data into the MySQL database
    insert_data_to_db(connection, filtered_data)

    # Close the connection after use
    connection.close()
    print("Data inserted and connection closed.")

# Run the main function
if __name__ == "__main__":
    main()


Connected to MySQL database
Data inserted and connection closed.
